In [1]:
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import time
from datetime import datetime, timedelta
import os.path

import nltk
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer 


from gensim.summarization.summarizer import summarize
from gensim import corpora
from gensim.models import LdaModel, LdaMulticore
from gensim.test.utils import datapath
from gensim.models import *
from gensim import similarities
from gensim.models import CoherenceModel
from gensim.matutils import softcossim


from GoogleNews import GoogleNews
googlenews = GoogleNews()

import newspaper
from newspaper import Article
from newspaper import fulltext

import pyLDAvis.gensim

from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

import random
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
style.use('ggplot')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import docx
from docx import Document
from docx.shared import RGBColor

from parsers import Google_parser as gp


## запрос по api
import requests
import json
from newsapi import NewsApiClient

import gensim.downloader as api
# Download the models
#fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')
word2vec_model300 = api.load('word2vec-google-news-300')
#glove_model300 = api.load('glove-wiki-gigaword-300')

from emailing import sendmailto


In [2]:
#Блок предобработки текста

stop_words = stopwords.words('english')
lemmatizator = nltk.stem.WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()


#функция предобработки
def prepare_text(text, stop = stop_words, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст 

    parameters:  
        stop: список из стоп-слов
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    #words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in tokens if item not in stop]
    words = [item for item in words if len(item)>3]
    
    return words #' '.join(words) #здесь выбираем формат вывода: в виде отдельных токенов или в форме связной строки-текста

def add_hyperlink(paragraph, url, text):
    """
    A function that places a hyperlink within a paragraph object.
    :param paragraph: The paragraph we are adding the hyperlink to.
    :param url: A string containing the required url
    :param text: The text displayed for the url
    :return: The hyperlink object
    """

    # This gets access to the document.xml.rels file and gets a new relation id value
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )

    # Create a w:r element
    new_run = docx.oxml.shared.OxmlElement('w:r')

    # Create a new w:rPr element
    rPr = docx.oxml.shared.OxmlElement('w:rPr')

    # Join all the xml elements together add add the required text to the w:r element
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)

    paragraph._p.append(hyperlink)

    return hyperlink

In [3]:
# слова-исключения, по которым будут отфильтровываться статьи
exclude_list = ['macbook','laptop','ultrabook','server', 'api', 'usb', 'installer', 'ubuntu', 'ip', 'lan','javascript', 'captcha', 'browser','internet', 'usb-c']  

### Выгрузки заголовков

In [4]:
transport_topic_list = ['maritime transport', 'port freight', 'container ship', 'bulk carrier', 'tanker', 'TEU', 'auto industry', 'refrigerated ship', 'reefer', 'supertanker', 'cargo ship']

agro_topic_list = ['china +red +line', 'china agriculture food subsidy', 'china +gmo', 'china food reserve', 'china agriculture policy', 'india fertilizer', 'monsoon el nino drought', 'brazil inflation real', 
              'argentina export tariff',  'EU food quota']

itt_topic_list = ['semiconductors market', '+software +market +volume', 'telecommunications market volume', 'iaas | paas | saas', '+it +market', 'computer sales', 'IT coronavirus']

In [5]:
# слова-исключения, по которым будут отфильтровываться статьи
exclude_list = ['macbook','laptop','ultrabook','server', 'api', 'usb', 'installer', 'ubuntu', 'ip', 'lan','javascript', 'captcha', 'browser','internet', 'usb-c', 'nintendo', 'docker', 'desktop', 'spacex', 'windows']  

In [6]:
def prepare_news_list_aggregate(topiclist, topicname):
    # задаем клиентский апи
    newsapi = NewsApiClient(api_key='8a38d185855d4fbda8c437e94149807d')
    # формируем границы периода времени, за которое будем запрашивать новости
    current_date = datetime.today()
    previous_day = current_date - timedelta(days = 1)
    current_date = current_date.strftime('%Y-%m-%d')
    previous_day = previous_day.strftime('%Y-%m-%d')

    articles_base_google = {}
    articles_base_newsapi = {}
    articles_base = {}
    for topic in topiclist:
        # выгрузка статей по одному запросу
        all_articles = []
        googlenews.clear()
        googlenews.search(topic)
        for item in googlenews.result()[0:5]:
            all_articles.append(item)

        articles_base_google[topic] = all_articles



        # получение данных по теме
        all_articles = newsapi.get_everything(q=topic,
                                              from_param=previous_day,
                                              to=current_date,
                                              language='en',
                                              sort_by='relevancy',
                                              page_size=5)


        articles_base_newsapi[topic] = all_articles['articles']

        articles_base[topic] = articles_base_google[topic] + articles_base_newsapi[topic]

    #формируем документ из коротких саммари новостей отфильтрованных на различность.
    document = Document()

    doc_header = 'Глобальные новости' + '\n' + 'Период: ' + str(previous_day) + ' - ' + str(current_date)
    document.add_heading(doc_header)

    for topic in articles_base:
        count = 1
        header = 'Новости по запросу: ' + str(topic)
        document.add_heading(header, level = 2)
        for item in articles_base[topic]:
            tokens = prepare_text(item['title'])
            raise_flag_excludeword = 0
            for token in tokens:
                if token in exclude_list:
                    raise_flag_excludeword = 1
            if raise_flag_excludeword == 0: 
                try:
                    url = item['url']
                except:
                    url = item['link']
                p = document.add_paragraph()
                link_text = str(count) + '. ' + item['title'] 
                url = add_hyperlink(p, url, link_text)
                count += 1

    doc_name = 'aggregate_' + str(topicname) + '_' + str(current_date) + '.docx'
    document.save(doc_name) 

In [7]:
prepare_news_list_aggregate(agro_topic_list, 'Agriculture')
prepare_news_list_aggregate(itt_topic_list, 'ITT')
prepare_news_list_aggregate(transport_topic_list, 'Transport')

### Блок транспортных новостей


In [ ]:
# задаем клиентский апи
newsapi = NewsApiClient(api_key='8a38d185855d4fbda8c437e94149807d')

# формируем границы периода времени, за которое будем запрашивать новости
current_date = datetime.today()
previous_day = current_date - timedelta(days = 1)
current_date = current_date.strftime('%Y-%m-%d')
previous_day = previous_day.strftime('%Y-%m-%d')


# transport_topiclist = [ 'maritime AND transport', 'port AND freight', '+cargo', 'transport AND delay', 'container AND ship', '+bulk', 'bulk AND carrier',
#                        '+tanker', '+TEU', 'auto AND industry', '+freight', 'refrigerated AND ship', '+reefer', '+supertanker', 'cargo AND ship']

transport_topiclist = [ '+maritime +transport', '+port +freight', '+cargo', '+container +ship', '+bulk', '+bulk +carrier',
                       '+tanker', '+TEU', '+auto +industry', '+freight', '+refrigerated +ship', '+reefer', '+supertanker', '+cargo +ship']
transport_all_articles = []
body_fulltext = []

for topic in transport_topiclist:
    #задаем тему запроса
    request_topic = topic

    # получение данных по теме
    all_articles = newsapi.get_everything(q=request_topic,
                                          from_param=previous_day,
                                          to=current_date,
                                          language='en',
                                          sort_by='relevancy',
                                          page_size=3)
    
    #Формирование служебных переменных
    count = 0
    error_link = [] #массив с битыми ссылками
    


    #замеряем время
    start_time = time.time()
    print('--- Выгружаются новости... ---')

    #попытаемся скачать текст новости из первоисточника, если не получится - оставим краткое описание
    for i in range(0, len(all_articles['articles'])):
        url  = all_articles['articles'][i]['url']
        try:
            html = requests.get(url).text
            text = fulltext(html)
            body_fulltext.append(text) 
            all_articles['articles'][i]['content'] = text 
        except:
            error_link.append(url) #иногда попадаются проблемные ссылки. Здесь мы будем сохранять их 
            pass
        count += 1
        if count % 5 == 0:
            print(count)
    
    print("--- Завершено. На выгрузку затрачено %s секунд ---" % (time.time() - start_time))
    transport_all_articles.append(all_articles)
#     #сохранение новостей в базу

#     save_path = 'C:/Users/Egran/YandexDisk/Work/news_project/news_database/'
#     new_filename = save_path + str(request_topic) + '_' + str(time.gmtime().tm_year) + '_' + str(time.gmtime().tm_mon) + '_' + str(time.gmtime().tm_mday) + '.txt'
#     with open(new_filename, 'w', encoding='utf-8') as f:
#         json.dump(all_articles, f, ensure_ascii=False, indent=4)
#         print('Выгрузка новостей сохранена в файл')
    

#здесь создаем три отдельных списка для текстов новостей, заголовков, ссылок. заполняем их
transport_all_contents = []
transport_all_titles = []
transport_all_urls = []
for article_list in transport_all_articles:
    i = 0
    for item in article_list['articles']:
        transport_all_contents.append(item['content'])
        transport_all_titles.append(item['title'])
        transport_all_urls.append(item['url'])

# проверяем можно ли записать вместо текстов саммари
for i in range (0, len(transport_all_contents)):
    try:
        transport_all_contents[i] = summarize(text = transport_all_contents[i], word_count = 100)
    except:
        pass

#формируем список списков из предобработанных саммари, разделенных на токены
#при формировании будем проверять наличие слов-исключений, связанных с посторонними темами.
error_index = 0
error_list = []
tokenized_list = []
for doc in transport_all_contents:
    try:
        tokens = prepare_text(doc)
        tokenized_list.append(tokens)
        error_index += 1
    except:
        error_list.append(error_index)
        error_index += 1
#отправляем это хозяйство в словарь и в корпус
mydict = corpora.Dictionary()
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_list]

# Prepare the similarity matrix
similarity_matrix = word2vec_model300.similarity_matrix(mydict, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

# конвертируем тексты новостей в bagofwords и создаем массив из списков bow
bag_of_words = []
for text in tokenized_list:
    bag_of_words.append(mydict.doc2bow(text))
    
# text_0 = mydict.doc2bow(tokenized_list[22])
# text_1 = mydict.doc2bow(tokenized_list[32])
     


# производится проверка на наличие схожих текстов. 
# если степень схожется меньше 0.5 проверяем на наличие текста в списке уникальных
# добавляем в список уникальных при условии несхожести и отсутствия дублирования
text_index = 0
comparison_index = 0
unique_articles = []
unique_urls = []
unique_titles = []

for text_index in range(0, len(bag_of_words)):
    try:
        comparison_index = 0
        similarity_flag = 0
        for comparison_index in range(text_index+1, len(bag_of_words)):
        #for comparison in bag_of_words:
            #if text_index != comparison_index:
            similarity_ratio = softcossim(bag_of_words[text_index], bag_of_words[comparison_index], similarity_matrix)
            if similarity_ratio > 0.2:
                similarity_flag = 1

        if (similarity_flag == 0) and (transport_all_contents[text_index] not in unique_articles):
            #текст потенциально подходит. проверим на наличие перекрестных тем
            #будем считать текст не по теме, если в нем встречается 2 и более посторонних тематических слов
            exclude_count = 0
            tokens = prepare_text(transport_all_contents[text_index])
            for word in tokens:
                if word in exclude_list:
                    exclude_count += 1
            if exclude_count == 0:
                unique_articles.append(transport_all_contents[text_index])
                unique_urls.append(transport_all_urls[text_index])
                unique_titles.append(transport_all_titles[text_index])
            else:
                pass
    except:
        pass
                
        
#формируем документ из коротких саммари новостей отфильтрованных на различность.
document = Document()
doc_datestamp = 'Период: ' + str(previous_day) + ' - ' + str(current_date)
doc_header = 'Новости транспорта'#'Тематика новостей: ' + str(request_topic)
document.add_heading(doc_header)
document.add_heading(doc_datestamp, level = 1)
for i in range (0, len(unique_articles)):
    #try:
        header = unique_titles[i]
        document.add_heading(header, level = 1)
        content = unique_articles[i]
        document.add_paragraph(content)
        url = unique_urls[i]
        p=document.add_paragraph()
        #p = document.add_paragraph()
        url = add_hyperlink(p, url, 'Ссылка на источник')
#         run = document.add_paragraph().add_run()
#         font = run.font
#         font.color.rgb = RGBColor(0x42, 0x24, 0xE9)
#         p=document.add_paragraph()
#         url = unique_urls[i]
#         document.add_paragraph(url)
        
    #except:
        #pass

#doc_name = str(request_topic) + '.docx' 
doc_name_transport = 'transport_' + str(current_date) + '.docx'
document.save(doc_name_transport)       

In [6]:
# sending mail
sendmailto('nikoda1@gmail.com', 'Global news', doc_datestamp, doc_name_transport)
sendmailto('Kirill.nikoda@gazprombank.ru', 'Global news', doc_datestamp, doc_name_transport)

### Блок Агро новостей


In [ ]:
# задаем клиентский апи
newsapi = NewsApiClient(api_key='8a38d185855d4fbda8c437e94149807d')

# формируем границы периода времени, за которое будем запрашивать новости
current_date = datetime.today()
previous_day = current_date - timedelta(days = 1)
current_date = current_date.strftime('%Y-%m-%d')
previous_day = previous_day.strftime('%Y-%m-%d')

topiclist = ['+agriculture AND +subsidy', '+china AND +gmo', '+agriculture AND +policy', '+agriculture AND +tax',
            '+food AND +trade', '+india AND +fertilizer', '+india AND +subsidy', '+china AND +subsidy', '+monsoon', 
             '+argentina AND +tariff', '+argentina AND +export', '+EU AND +quota', '+el AND +nino', '+usa AND +drought']
# topiclist = ['+china AND red AND line', '+agriculture AND +subsidy', '+china AND +gmo', '+agriculture AND +policy', '+agriculture AND +tax',
#             '+food AND +trade', '+india AND +fertilizer', '+india AND +subsidy', '+china AND +subsidy', '+monsoon', '+brazil AND +inflation', '+brazil AND +real', 
#              '+argentina AND +tariff', '+argentina AND +export', '+EU AND +quota', '+el +nino', '+usa AND +drought']
# topiclist = ['china AND land', 'china AND red AND line', 'agro AND subsidy', 'china AND gmo', 'food AND reserve', 'agriculture AND policy', 'agriculture AND tax',
#             'china AND tax', 'food AND trade', 'india AND fertilizer', 'india AND subsidy', 'china AND subsidy', 'monsoon', 'brazil AND inflation', 'brazil AND real', 
#              'argentina AND tariff', 'argentina AND export', 'EU AND quota', 'eu AND food', 'el nino', 'usa AND drought']
all_articles = []
body_fulltext = []

for topic in topiclist:
    #задаем тему запроса
    request_topic = topic

    # получение данных по теме
    articles = newsapi.get_everything(q = request_topic,
                                          from_param=previous_day,
                                          to=current_date,
                                          language='en',
                                          sort_by='relevancy',
                                          page_size=3)
    
    #Формирование служебных переменных
    count = 0
    error_link = [] #массив с битыми ссылками
    


    #замеряем время
    start_time = time.time()
    print('--- Выгружаются новости... ---')

    #попытаемся скачать текст новости из первоисточника, если не получится - оставим краткое описание
    for i in range(0, len(articles['articles'])):
        url  = articles['articles'][i]['url']
        try:
            html = requests.get(url).text
            text = fulltext(html)
            body_fulltext.append(text) 
            articles['articles'][i]['content'] = text 
        except:
            error_link.append(url) #иногда попадаются проблемные ссылки. Здесь мы будем сохранять их 
            pass
        count += 1
        if count % 5 == 0:
            print(count)
    
    print("--- Завершено. На выгрузку затрачено %s секунд ---" % (time.time() - start_time))
    all_articles.append(articles)
#     #сохранение новостей в базу

#     save_path = 'C:/Users/Egran/YandexDisk/Work/news_project/news_database/'
#     new_filename = save_path + str(request_topic) + '_' + str(time.gmtime().tm_year) + '_' + str(time.gmtime().tm_mon) + '_' + str(time.gmtime().tm_mday) + '.txt'
#     with open(new_filename, 'w', encoding='utf-8') as f:
#         json.dump(all_articles, f, ensure_ascii=False, indent=4)
#         print('Выгрузка новостей сохранена в файл')
    

#здесь создаем три отдельных списка для текстов новостей, заголовков, ссылок. заполняем их
all_contents = []
all_titles = []
all_urls = []
for article_list in all_articles:
    i = 0
    for item in article_list['articles']:
        all_contents.append(item['content'])
        all_titles.append(item['title'])
        all_urls.append(item['url'])

# проверяем можно ли записать вместо текстов саммари
for i in range (0, len(all_contents)):
    try:
        all_contents[i] = summarize(text = all_contents[i], word_count = 100)
    except:
        pass

#формируем список списков из предобработанных саммари, разделенных на токены
#при формировании будем проверять наличие слов-исключений, связанных с посторонними темами.
error_index = 0
error_list = []
tokenized_list = []
for doc in all_contents:
    try:
        tokens = prepare_text(doc)
        tokenized_list.append(tokens)
        error_index += 1
    except:
        error_list.append(error_index)
        error_index += 1
#отправляем это хозяйство в словарь и в корпус
mydict = corpora.Dictionary()
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_list]

# Prepare the similarity matrix
similarity_matrix = word2vec_model300.similarity_matrix(mydict, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

# конвертируем тексты новостей в bagofwords и создаем массив из списков bow
bag_of_words = []
for text in tokenized_list:
    bag_of_words.append(mydict.doc2bow(text))
          


# производится проверка на наличие схожих текстов. 
# если степень схожется меньше 0.5 проверяем на наличие текста в списке уникальных
# добавляем в список уникальных при условии несхожести и отсутствия дублирования
text_index = 0
comparison_index = 0
unique_articles = []
unique_urls = []
unique_titles = []

for text_index in range(0, len(bag_of_words)):
    comparison_index = 0
    similarity_flag = 0
    for comparison_index in range(text_index+1, len(bag_of_words)):
    #for comparison in bag_of_words:
        #if text_index != comparison_index:
        similarity_ratio = softcossim(bag_of_words[text_index], bag_of_words[comparison_index], similarity_matrix)
        if similarity_ratio > 0.2:
            similarity_flag = 1

    if (similarity_flag == 0) and (all_contents[text_index] not in unique_articles):
        #текст потенциально подходит. проверим на наличие перекрестных тем
        #будем считать текст не по теме, если в нем встречается 2 и более посторонних тематических слов
        exclude_count = 0
        tokens = prepare_text(all_contents[text_index])
        for word in tokens:
            if word in exclude_list:
                exclude_count += 1
        if exclude_count == 0:
            unique_articles.append(all_contents[text_index])
            unique_urls.append(all_urls[text_index])
            unique_titles.append(all_titles[text_index])
        else:
            pass

        #comparison_index += 1
    #text_index += 1
                
        
#формируем документ из коротких саммари новостей отфильтрованных на различность.
document = Document()
doc_datestamp = 'Период: ' + str(previous_day) + ' - ' + str(current_date)
doc_header = 'Новости агросектора'#'Тематика новостей: ' + str(request_topic)
document.add_heading(doc_header)
document.add_heading(doc_datestamp, level = 1)
for i in range (0, len(unique_articles)):
    #try:
        header = unique_titles[i]
        document.add_heading(header, level = 1)
        content = unique_articles[i]
        document.add_paragraph(content)
        url = unique_urls[i]
        p=document.add_paragraph()
        #p = document.add_paragraph()
        url = add_hyperlink(p, url, 'Ссылка на источник')
#         run = document.add_paragraph().add_run()
#         font = run.font
#         font.color.rgb = RGBColor(0x42, 0x24, 0xE9)
#         p=document.add_paragraph()
#         url = unique_urls[i]
#         document.add_paragraph(url)
        
    #except:
        #pass

#doc_name = str(request_topic) + '.docx' 
doc_name_agro = 'agro_' + str(current_date) + '.docx'
document.save(doc_name_agro)       

In [ ]:
sendmailto('snitkodv@mail.ru', 'Global news', doc_datestamp, doc_name_agro)
sendmailto('Daria.Snitko@gazprombank.ru', 'Global news', doc_datestamp, doc_name_agro)

### Energy

In [ ]:
# задаем клиентский апи
newsapi = NewsApiClient(api_key='8a38d185855d4fbda8c437e94149807d')

# формируем границы периода времени, за которое будем запрашивать новости
current_date = datetime.today()
previous_day = current_date - timedelta(days = 1)
current_date = current_date.strftime('%Y-%m-%d')
previous_day = previous_day.strftime('%Y-%m-%d')

topiclist = ['+china' ]
# topiclist = ['+china AND red AND line', '+agriculture AND +subsidy', '+china AND +gmo', '+agriculture AND +policy', '+agriculture AND +tax',
#             '+food AND +trade', '+india AND +fertilizer', '+india AND +subsidy', '+china AND +subsidy', '+monsoon', '+brazil AND +inflation', '+brazil AND +real', 
#              '+argentina AND +tariff', '+argentina AND +export', '+EU AND +quota', '+el +nino', '+usa AND +drought']
# topiclist = ['china AND land', 'china AND red AND line', 'agro AND subsidy', 'china AND gmo', 'food AND reserve', 'agriculture AND policy', 'agriculture AND tax',
#             'china AND tax', 'food AND trade', 'india AND fertilizer', 'india AND subsidy', 'china AND subsidy', 'monsoon', 'brazil AND inflation', 'brazil AND real', 
#              'argentina AND tariff', 'argentina AND export', 'EU AND quota', 'eu AND food', 'el nino', 'usa AND drought']
all_articles = []
body_fulltext = []

for topic in topiclist:
    #задаем тему запроса
    request_topic = topic

    # получение данных по теме
    articles = newsapi.get_everything(q = request_topic,
                                          from_param=previous_day,
                                          to=current_date,
                                          language='en',
                                          sort_by='relevancy',
                                          page_size=3)
    
    #Формирование служебных переменных
    count = 0
    error_link = [] #массив с битыми ссылками
    


    #замеряем время
    start_time = time.time()
    print('--- Выгружаются новости... ---')

    #попытаемся скачать текст новости из первоисточника, если не получится - оставим краткое описание
    for i in range(0, len(articles['articles'])):
        url  = articles['articles'][i]['url']
        try:
            html = requests.get(url).text
            text = fulltext(html)
            body_fulltext.append(text) 
            articles['articles'][i]['content'] = text 
        except:
            error_link.append(url) #иногда попадаются проблемные ссылки. Здесь мы будем сохранять их 
            pass
        count += 1
        if count % 5 == 0:
            print(count)
    
    print("--- Завершено. На выгрузку затрачено %s секунд ---" % (time.time() - start_time))
    all_articles.append(articles)
#     #сохранение новостей в базу

#     save_path = 'C:/Users/Egran/YandexDisk/Work/news_project/news_database/'
#     new_filename = save_path + str(request_topic) + '_' + str(time.gmtime().tm_year) + '_' + str(time.gmtime().tm_mon) + '_' + str(time.gmtime().tm_mday) + '.txt'
#     with open(new_filename, 'w', encoding='utf-8') as f:
#         json.dump(all_articles, f, ensure_ascii=False, indent=4)
#         print('Выгрузка новостей сохранена в файл')
    

#здесь создаем три отдельных списка для текстов новостей, заголовков, ссылок. заполняем их
all_contents = []
all_titles = []
all_urls = []
for article_list in all_articles:
    i = 0
    for item in article_list['articles']:
        all_contents.append(item['content'])
        all_titles.append(item['title'])
        all_urls.append(item['url'])

# проверяем можно ли записать вместо текстов саммари
for i in range (0, len(all_contents)):
    try:
        all_contents[i] = summarize(text = all_contents[i], word_count = 100)
    except:
        pass

#формируем список списков из предобработанных саммари, разделенных на токены
#при формировании будем проверять наличие слов-исключений, связанных с посторонними темами.
error_index = 0
error_list = []
tokenized_list = []
for doc in all_contents:
    try:
        tokens = prepare_text(doc)
        tokenized_list.append(tokens)
        error_index += 1
    except:
        error_list.append(error_index)
        error_index += 1
#отправляем это хозяйство в словарь и в корпус
mydict = corpora.Dictionary()
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_list]

# Prepare the similarity matrix
similarity_matrix = word2vec_model300.similarity_matrix(mydict, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

# конвертируем тексты новостей в bagofwords и создаем массив из списков bow
bag_of_words = []
for text in tokenized_list:
    bag_of_words.append(mydict.doc2bow(text))
          


# производится проверка на наличие схожих текстов. 
# если степень схожется меньше 0.5 проверяем на наличие текста в списке уникальных
# добавляем в список уникальных при условии несхожести и отсутствия дублирования
text_index = 0
comparison_index = 0
unique_articles = []
unique_urls = []
unique_titles = []

for text_index in range(0, len(bag_of_words)):
    comparison_index = 0
    similarity_flag = 0
    for comparison_index in range(text_index+1, len(bag_of_words)):
    #for comparison in bag_of_words:
        #if text_index != comparison_index:
        similarity_ratio = softcossim(bag_of_words[text_index], bag_of_words[comparison_index], similarity_matrix)
        if similarity_ratio > 0.2:
            similarity_flag = 1

    if (similarity_flag == 0) and (all_contents[text_index] not in unique_articles):
        #текст потенциально подходит. проверим на наличие перекрестных тем
        #будем считать текст не по теме, если в нем встречается 2 и более посторонних тематических слов
        exclude_count = 0
        tokens = prepare_text(all_contents[text_index])
        for word in tokens:
            if word in exclude_list:
                exclude_count += 1
        if exclude_count == 0:
            unique_articles.append(all_contents[text_index])
            unique_urls.append(all_urls[text_index])
            unique_titles.append(all_titles[text_index])
        else:
            pass

        #comparison_index += 1
    #text_index += 1
                
        
#формируем документ из коротких саммари новостей отфильтрованных на различность.
document = Document()
doc_datestamp = 'Период: ' + str(previous_day) + ' - ' + str(current_date)
doc_header = 'Новости энергетики'#'Тематика новостей: ' + str(request_topic)
document.add_heading(doc_header)
document.add_heading(doc_datestamp, level = 1)
for i in range (0, len(unique_articles)):
    #try:
        header = unique_titles[i]
        document.add_heading(header, level = 1)
        content = unique_articles[i]
        document.add_paragraph(content)
        url = unique_urls[i]
        p=document.add_paragraph()
        #p = document.add_paragraph()
        url = add_hyperlink(p, url, 'Ссылка на источник')
#         run = document.add_paragraph().add_run()
#         font = run.font
#         font.color.rgb = RGBColor(0x42, 0x24, 0xE9)
#         p=document.add_paragraph()
#         url = unique_urls[i]
#         document.add_paragraph(url)
        
    #except:
        #pass

#doc_name = str(request_topic) + '.docx' 
doc_name = 'energy_' + str(current_date) + '.docx'
document.save(doc_name)       